In [ ]:
using JuMP
#Import HiGHS solver
using HiGHS

#Create a JuMP model named picframe1 that will be solved using the HiGHS solver
coffeeblend = Model(HiGHS.Optimizer);
types = [:Columbian,:Brazilian,:Sumatran];
blends = [:Robust, :Light];

beansAviliable = Dict(:Columbian=> 550,:Brazilian=>450,:Sumatran=>650);
beansPrice=Dict(:Columbian=> 1,:Brazilian=>0.85,:Sumatran=>1.55);
coffPrice=Dict(:Robust=>4.25,:Light=>3.95)

#inf represent lowerbound; sup represent upper bound; setting upper lower bound for beans in each coffee
@variable(coffeeblend,x[types, blends]>=0);
@constraint(coffeeblend, RSInf, x[:Sumatran,:Robust] >= 0.6*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, RSSup, x[:Sumatran,:Robust] <= 0.75*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, RCInf, x[:Columbian,:Robust] >= 0.1*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, LBInf, x[:Brazilian,:Light] >= 0.5*sum(x[i,:Light] for i in types));
@constraint(coffeeblend, LBSup, x[:Brazilian,:Light] <= 0.6*sum(x[i,:Light] for i in types));
@constraint(coffeeblend, LSSup, x[:Sumatran,:Light] <= 0.6*sum(x[i,:Light] for i in types));
#setting bound for beans available
@constraint(coffeeblend, beansConstraint[i in types], sum(x[i,j] for j in blends)<=beansAviliable[i]);

#objective function with revenue-cost
@objective(coffeeblend,Max, sum(x[i,j] for i in types for j in blends)*coffPrice[j]-
sum(x[i,j] for i in types for j in blends)*beansPrice[i]);

print(coffeeblend);


In [ ]:
optimize!(coffeeblend);
@show solution_summary(coffeeblend);
@show objective_value(coffeeblend)